In [55]:
import numpy as np
import math
from scipy.optimize import linprog

In [56]:
# Ax <= b
def inner_point(A, b):
    n = len(A[0])
    cur_b = [(-np.inf, np.inf) for i in range(n)]
    def solve(c):
        return linprog(c, A_ub=A, b_ub=b, bounds=cur_b)
    
    #middle point of coordinate bounds
    def step_middle(i):
        res_min = solve(np.eye(1, n, i)[0])
        res_max = solve(-np.eye(1, n, i)[0])
        
        status = (res_min.status, res_max.status)
        if status == (3, 3):
            return 0
        elif status == (3, 0):
            return -res_max.fun - 1
        elif status == (0, 3):
            return res_min.fun + 1
        elif status == (0, 0):
            is_equal = (res_min.fun + res_max.fun) / 2
            res = (res_min.fun - res_max.fun) / 2
            return None if is_equal == 0 else res
        else:
            return None
    
    ans = []
    for i in range(n):
        x_i = step_middle(i)
        if x_i is None:
            return None
        ans.append(x_i)
        cur_b[i] = (x_i, x_i)
    return ans

In [61]:
N = 70

def inner_point_method(A, b, c, Ag, bg, x_start = None):
    n = len(Ag[0])
    m = len(Ag)

    def func(x):
        return (0.5 * np.matmul(np.transpose(x), np.matmul(A, x)) - np.matmul(np.transpose(b), x) + c)[0][0]
    
    def f_grad(x):
        return np.matmul(A, x) - b

    def f_hess(x):
        return A
    
    def log_bar_grad(v):
        s = 0
        for i in range(len(bg)):
            s += 1.0 / (bg[i][0] - np.dot(Ag[i], v)[0]) * Ag[i].T
        return np.matrix(s).T

    def log_bar_hess(v):
        s = 0
        d = np.diag(np.array([1.0 / (bg[i][0] - np.matmul(Ag[i], v)[0]) ** 2 for i in range(m)]))
        return np.matmul(Ag.T, np.matmul(d, Ag))

    def F(x, t):
        return t * func(x) - sum([np.log(bg[i][0] - np.matmul(Ag[i], x)[0]) for i in range(m)])
    
    def F_grad(x, t):
        return t * f_grad(x) + log_bar_grad(x)
    
    def F_hess(x, t):
        return t * f_hess(x) + log_bar_hess(x)
    
    alpha = 1.1
    t = 0.5
    traj = []
    traj.append(x_start.copy())
    cur_x = x_start.copy()
    for i in range(N):
        cur_x = np.array(cur_x - np.matmul(np.linalg.inv(F_hess(cur_x, t)), F_grad(cur_x, t)))
        traj.append(cur_x.copy())
        t *= alpha
    return traj

In [62]:
A = np.array([[4, -2], [-2, 2]])
b = np.array([[1], [2]])
c = 3
Ag = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
bg = np.array([[1], [1], [1], [1]])
traj = inner_point_method(A, b, c, Ag, bg, np.array(np.matrix(inner_point(Ag, bg)).T))
print(traj[-1])
#ans
print(np.array([[3.0 / 4], [1]]))

[[0.74672722]
 [0.99815644]]
[[0.75]
 [1.  ]]
